# Исследование надежности заемщиков

**Заказчик** — кредитный отдел банка. 

**Задача** — разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

**Входные данные от банка** — статистика о платёжеспособности клиентов.

- Результаты исследования будут учтены при построении модели кредитного скоринга — оценки способности потенциального заёмщика вернуть кредит банку.

## Обзор данных

In [1]:
import pandas as pd

In [2]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Данные имеют пропуски, категорийные признаки education, purpose требую доработки

## Предобработка данных

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Заполним пропуски в total_income мерианным значением для этой группы

In [6]:
for t in data['income_type'].unique():
    (data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 
     'total_income']) = data.loc[(data['income_type'] == t), 'total_income'].median()

Заменим все отрицательные значения положительными в столбце `'days_employed'`

In [7]:
data['days_employed'] = data['days_employed'].abs()

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения, оставим их как есть.

Рассмотрим уникальные значения в столбце `'children'`

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Исключим из рассмотрения строки со значением `'children'` -1 и 20

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Заполним пропуски в столбце `'days_employed'` медианными значениями по каждому типу занятости `'income_type'`. 

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Все пропуски заполнены. Приведем данные в столбце `'total_income'` в целочисленный тип.

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце education. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру и удалим полные дубликаты. 

In [15]:
data['education'] = data['education'].str.lower()

In [16]:
data.duplicated().sum()

71

In [17]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме столбец `'total_income_category'` с категориями
- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Рассмотрим уникальных целей взятия кредита из столбца `'purpose'`.

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Категиризируем данные из столбца `'purpose'` сформирует новый столбец `'purpose_category'`, на следующие категории
- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Проверка гипотез

### Зависимость между количеством детей и возвратом кредита в срок

Определим средний процент клиентов допустивших просрочку по платежу:

In [23]:
rows_number = data.shape[0]
debt_sum = data.loc[data.loc[:,'debt'] == 1]['debt'].count()
debt_all_mean = debt_sum / rows_number
(f'Средний процент клиентов допустивших просрочку по платежу: {debt_all_mean:.2%}')

'Средний процент клиентов допустивших просрочку по платежу: 8.12%'

Сделаем сводную таблицу по количеству детей и допущенным просрочкам. Ознакомимся с таблицей. 

In [24]:
data_pivot_children = data.pivot_table(index=['children'], columns='debt', 
                                       values='purpose_category', aggfunc='count')
data_pivot_children = data_pivot_children.rename(columns={0: 'no_debt', 1: 'have_debt'})
data_pivot_children['have_debt'] = data_pivot_children['have_debt'].fillna(value=0) 
display(data_pivot_children) 

debt,no_debt,have_debt
children,,
0,13028.0,1063.0
1,4364.0,444.0
2,1858.0,194.0
3,303.0,27.0
4,37.0,4.0
5,9.0,0.0


В сводной таблице по количеству детей и допущенным просрочкам расчитаем общее количество клиентов по каждой категории и их доли.
А также расчитаем **среднее значение просрочек по категории - mean**.

In [25]:
data_pivot_children['total'] = (data_pivot_children['no_debt'] 
                                + data_pivot_children['have_debt'])
data_pivot_children['part'] = data_pivot_children['total'] * 100 / rows_number
try:
    data_pivot_children['mean'] = (data_pivot_children['have_debt'] *100 
                                   /  data_pivot_children['total'])
except:
    data_pivot_children['mean'] = None
data_pivot_children.head(10)

debt,no_debt,have_debt,total,part,mean
children,,,,,
0,13028.0,1063.0,14091.0,66.058788,7.543822
1,4364.0,444.0,4808.0,22.539965,9.234609
2,1858.0,194.0,2052.0,9.619802,9.454191
3,303.0,27.0,330.0,1.547044,8.181818
4,37.0,4.0,41.0,0.192209,9.756098
5,9.0,0.0,9.0,0.042192,0.000000


**Вывод:** 

Да есть зависимость между количеством детей и возвратом кредита в срок.

- Категория "без детей" доля - 66.1% имеет значение просрочек 7.5%. Что на **0,6% лучше** среднего.
- Категория "с 1-м ребенком" доля - 22.4% имеет значение просрочек 9.2%. Что на **1,1% ХУЖЕ** среднего.
- Категория "с 2-мя детьми" доля - 9.6% имеет значение просрочек 9.5%. Что на **1,4% ХУЖЕ** среднего.
- Категория "с 3-мя детьми" доля - 1.5% имеет значение просрочек 8.2%. Что совпадает со средним. По данной категории требуются дополнительные данные.
- Категория "с 4-мя детьми" требуются дополнительные данные.
- Категория "с 5-ю детьми" требуются дополнительные данные.

Наблюдается резкий рост между категорией "без детей" и "с 1-м ребенком". Далее идет практически линейный рост просрочек с увеличением количества детей. Однозначно это можно утвержать для категорий с 1-м и 2-мя детьми.

В целом с **увеличением количества детей у клиента вероятность просрочки возрастает.**

### Зависимость между семейным положением и возвратом кредита в срок

Проверим корректность заполнения столбцов `'family_status_id'` и `'family_status'`. Создаем сводную таблицу.

In [26]:
data_pivot_family_id = data.pivot_table(index=['family_status_id'], 
                                        columns='family_status', 
                                        values='purpose_category', 
                                        aggfunc='count')
data_pivot_family_id

family_status,Не женат / не замужем,в разводе,вдовец / вдова,гражданский брак,женат / замужем
family_status_id,,,,,
0,NaN,NaN,NaN,NaN,12261.0
1,NaN,NaN,NaN,4134.0,NaN
2,NaN,NaN,951.0,NaN,NaN
3,NaN,1189.0,NaN,NaN,NaN
4,2796.0,NaN,NaN,NaN,NaN


Столбцы `'family_status_id'` и `'family_status'` заполненны корректно, в дальнейшем используем столбец `'family_status'`.

Сделаем сводную таблицу по семейному положению и допущенным просрочкам. Ознакомимся с таблицей.

In [27]:
data_pivot_family = data.pivot_table(index=['family_status'], 
                                     columns='debt', 
                                     values='purpose_category', 
                                     aggfunc='count')
data_pivot_family = data_pivot_family.rename(columns={0: 'no_debt', 
                                                      1: 'have_debt'}) 
display(data_pivot_family)

debt,no_debt,have_debt
family_status,,
Не женат / не замужем,2523,273
в разводе,1105,84
вдовец / вдова,888,63
гражданский брак,3749,385
женат / замужем,11334,927


В сводной таблице по семейному положению и допущенным просрочкам расчитаем общее количество клиентов по каждой категории 
и их доли. А также расчитаем **среднее значение просрочек по категории - mean.**

In [28]:
data_pivot_family['total'] =(data_pivot_family['no_debt'] 
                             + data_pivot_family['have_debt'])
data_pivot_family['part'] = data_pivot_family['total'] * 100 / rows_number
try:
    data_pivot_family['mean'] = (data_pivot_family['have_debt'] * 100
                                 / data_pivot_family['total'])
except:
    data_pivot_family['mean'] = None
display(data_pivot_family)

debt,no_debt,have_debt,total,part,mean
family_status,,,,,
Не женат / не замужем,2523,273,2796,13.107684,9.763948
в разводе,1105,84,1189,5.574047,7.064760
вдовец / вдова,888,63,951,4.458300,6.624606
гражданский брак,3749,385,4134,19.380245,9.313014
женат / замужем,11334,927,12261,57.479724,7.560558


**Вывод:** 

Да есть зависимость между семейным положением и возвратом кредита в срок.

- Наибольшая доля клиентов 57.5% - категория "женат / замужем" имеет значение просрочек 7.6% что **лучше** среднего.
- Категория "вдовец / вдова" доля - 4.5% имеет значение просрочек 6.6%. Что на **1,5% лучше** среднего.
- Категория "в разводе" доля - 5.6% имеет значение просрочек 7.1%. Что на **1,0% лучше** среднего.
- Категория "гражданский брак" доля - 19.4% имеет значение просрочек 9.3%. Что на **1,2% ХУЖЕ** среднего.
- Категория "Не женат / не замужем " доля - 13.1% имеет значение просрочек 9.8%. Что на **1,7% ХУЖЕ** среднего.

**Клиенты состоящие или состоявшие в браке реже допускают просрочку!**

### Зависимость между уровнем дохода и возвратом кредита в срок

Сделаем сводную таблицу по категории дохода и допущенным просрочкам. Ознакомимся с таблицей.

In [29]:
data_pivot_income = data.pivot_table(index=['total_income_category'], 
                                     columns='debt', 
                                     values='purpose_category', 
                                     aggfunc='count')
data_pivot_income = data_pivot_income.rename(columns={0: 'no_debt', 
                                                      1: 'have_debt'}) 
display(data_pivot_income)

debt,no_debt,have_debt
total_income_category,,
A,23,2
B,4660,354
C,14568,1353
D,328,21
E,20,2


В сводной таблице по категории дохода и допущенным просрочкам расчитаем общее количество клиентов по каждой категории 
и их доли. А также расчитаем **среднее значение просрочек по категории - mean.**

In [30]:
data_pivot_income['total'] = (data_pivot_income['no_debt'] 
                              + data_pivot_income['have_debt'])
data_pivot_income['part'] = data_pivot_income['total'] * 100 / rows_number
try:
    data_pivot_income['mean'] = (data_pivot_income['have_debt'] *100 
                                 / data_pivot_income['total'])
except:
    data_pivot_income['mean'] = None
display(data_pivot_income)

debt,no_debt,have_debt,total,part,mean
total_income_category,,,,,
A,23,2,25,0.117200,8.000000
B,4660,354,5014,23.505696,7.060231
C,14568,1353,15921,74.637851,8.498210
D,328,21,349,1.636116,6.017192
E,20,2,22,0.103136,9.090909


**Вывод:** 
Да есть зависимость между уровнем дохода и возвратом кредита в срок.

- Категория "A" - имеется всего 25 наблюдений, данных недостаточно. На сегодня- показатели средние.
- Категория "B" - доля 23.5%. имеет значение просрочек 7.1%. Что на **1,0% лучше** среднего.
- Категория "C" - доля 74.6%. имеет значение просрочек 8.5%. Что на **0,4% ХУЖЕ** среднего.
- Категория "D" - доля 1.6%. имеет значение просрочек 6.0%. Что на **2,1% лучше** среднего. Нужны дололнительные данные.
- Категория "E" - имеется всего 22 наблюдения, данных недостаточно. На сегодня- показатели хуже средних.

В категориях "B" и "C" имеются достаточные ряды наблюдений. Выявлена прямая зависимость между доходом клиента и возвратом кредита в срок. 

**Чем больше доход клиента тем реже он допускает просрочки.** 

### Влияние цели кредита на его возврат в срок

Сделаем сводную таблицу по цели кредита и допущенным просрочкам. Ознакомимся с таблицей.

In [31]:
data_pivot_purpose = data.pivot_table(index=['purpose_category'], 
                                      columns='debt', 
                                      values='total_income_category', 
                                      aggfunc='count')
data_pivot_purpose = data_pivot_purpose.rename(columns={0: 'no_debt', 
                                                        1: 'have_debt'}) 
display(data_pivot_purpose)

debt,no_debt,have_debt
purpose_category,,
операции с автомобилем,3879,400
операции с недвижимостью,9971,780
получение образования,3619,369
проведение свадьбы,2130,183


В сводной таблице по цели кредита и допущенным просрочкам расчитаем общее количество клиентов по каждой категории и их доли. А также расчитаем среднее значение просрочек по категории - mean.

In [32]:
data_pivot_purpose['total'] = (data_pivot_purpose['no_debt'] 
                               + data_pivot_purpose['have_debt'])
data_pivot_purpose['part'] = data_pivot_purpose['total'] * 100 / rows_number
try:
    data_pivot_purpose['mean'] = (data_pivot_purpose['have_debt'] *100 
                                  / data_pivot_purpose['total'])
except:
    data_pivot_purpose['mean'] = None
display(data_pivot_purpose)

debt,no_debt,have_debt,total,part,mean
purpose_category,,,,,
операции с автомобилем,3879,400,4279,20.060007,9.347978
операции с недвижимостью,9971,780,10751,50.400825,7.255139
получение образования,3619,369,3988,18.695795,9.252758
проведение свадьбы,2130,183,2313,10.843373,7.911803


**Вывод:**
Да есть зависимость между целью кредита и возвратом кредита в срок.

- "Операции с недвижимостью" с долей 50.4% имеют долю просрочек 7.3%. Что на **0.8% лучше** среднего.
- "Проведение свадьбы" с долей 10.8% имеют долю просрочек 7.9%.
- "Получение образования" с долей 18.7%  и "Операции с автомобилем" с долей 20.1%  имеют долю просрочек 9.3%. Что на **1.2% ХУЖЕ** среднего.

По займам на **"Операции с недвижимостью" и "Проведение свадьбы" реже допускаются просрочки чем на "Получение образования"  и "Операции с автомобилем".**

## Общий вывод

В ходе предобработки данных были выполнены следующие действия:
*  Пропуски в столбеце "total_income", который хранит данные о доходах, были заполнены медианными значениями с учетом типа занятости. Пропуски составляют ~ 10% от выборки что значительно и составляет почти половину от численности категории "B" по доходу. Это могло вовлиять на деление клиентов по категориям дохода. *(Средний уровень просрочек у строк с пропусками и без равен среднему по выборке)*  
* Отрицательные значения "days_employed" - трудового стажа были заменены на абсолютные. Пропуски в столбеце, были заполнены медианными значениями с учетом типа занятости. Пропуски составляют ~ 10% от выборки.  Также в столбце есть аномальные значения показывающие стаж более 950 лет на клиента. На текущее исследование это не влияет.
* Строки с аномальными значениями в столбце "children" были удалены. Удаленные строки составляют 0.9% выборки. На результаты исследования это не повлияло.
* Столбец'education' очищен от дубликатов приведением к одинаковому регистру.
* После этих манипуляций выявлен и удален 71 полный дубликат что соствляет 0.3% выборки.
* Выполнена категоризация по уровню дохода на 5 категорий. И по цели займа на 4 категории.
* Средний уровень просрочки: 8.12%

Результаты: 
* Есть прямая зависимость просрочек от наличия и количества детей у клиента. Для бездетных уровень просрочки 7.5%, для 1-го ребенка уже 9.2% с последующим ростом.
* Есть зависимость между семейным положением: состоящие или состоявшие в браке ~ на 2% реже допускают просрочку.
* Выявлена прямая зависимость между доходом клиента и возвратом кредита в срок. Чем больше доход клиента тем стабильнее платежи. Это справедливо для диапазона 50 000 - 1 000 000.
* По займам на "Операции с недвижимостью" и "Проведение свадьбы"  ~ на 2% реже допускаются просрочки чем на "Получение образования" и "Операции с автомобилем".

Дополнительные наблюдения:
*  По имеющимся на сегодня данным, с точки зрения своевременности платежей, категория "D" с  уровенем просрочек 6.0% - лучшая из всех групп. В то же время данных по ней недостаточно, нужен эксперимент и мониторинг.